<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/verbs_cosine_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz


--2020-02-19 09:47:27--  http://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549952184 (524M) [application/x-gzip]
Saving to: ‘news_mystem_skipgram_1000_20_2015.bin.gz’

news_mystem_skipgra 100%[===================>] 524.47M  27.5MB/s    in 20s     

2020-02-19 09:47:47 (26.1 MB/s) - ‘news_mystem_skipgram_1000_20_2015.bin.gz’ saved [549952184/549952184]



In [3]:
!rm collection.json.1 collection.json
!wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json

rm: cannot remove 'collection.json.1': No such file or directory
rm: cannot remove 'collection.json': No such file or directory
--2020-02-19 09:47:50--  https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 606414 (592K) [text/plain]
Saving to: ‘collection.json’

collection.json     100%[===================>] 592.20K  --.-KB/s    in 0.06s   

2020-02-19 09:47:50 (8.96 MB/s) - ‘collection.json’ saved [606414/606414]



In [0]:
import gensim
from collections import Counter
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

In [5]:
rv_model = gensim.models.KeyedVectors.load_word2vec_format('news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# rv_model.vocab
# rv_model.get_vector('помочь_V')

In [7]:

verbs_for_check_dict = {'снимать': [['a0', 'a1', 'neg', 1.0]]}

def just_words_similarity(base_verb, verbs_for_check_dict):
  """
  Функция тестовая, проверяет, как работает косинусная близость.
  """
  group_dict = Counter()
  my_base_verb = base_verb + '_V'
  for verb in verbs_for_check_dict.keys():
    verb_for_check = verb + '_V'
    similarity = rv_model.similarity(my_base_verb, verb_for_check)
    group_dict[str(verbs_for_check_dict[verb])] = [base_verb, verb, similarity]
  return group_dict

just_words_similarity('радоваться', verbs_for_check_dict)




/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Counter({"[['a0', 'a1', 'neg', 1.0]]": ['радоваться', 'снимать', 0.15058792]})

In [0]:
# rv_model.vocab

In [0]:
collection_handler = open("collection.json", "r", encoding="utf-8")
collection = json.load(collection_handler, encoding="utf-8")

In [28]:

def make_group(polarity_type_list):
  """
  На вход подаются пары из разных типов отношений(p_type) в списках(как в переменной pol_type_dict)
  для каждой пары цикл ищет не более ста глаголов, у которых есть типы отношений, совпадающие с нашей парой, 
  которая подается на вход. 
  """
  group_dict = Counter()
  for p_type in polarity_type_list:
    print(p_type)
    count = 0
    for key in collection.keys():
      if count < 100:
        try:
          #print(collection[key]['frames']['polarity'])
          if p_type[0] in collection[key]['frames']['polarity'] and p_type[1] in collection[key]['frames']['polarity'] :
          # эту строчку можно менять, если нам нужно искать отношения, где только ['a0', 'a1', 'pos', 1.0]:
          # if p_type[0] in collection[key]['frames']['polarity'] and p_type[1] not in collection[key]['frames']['polarity'] : 
          # проблема! в лексиконе всего 20 отношений [['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]] c разными коэффициентами, и 51 таких, 
          # где ['a0', 'a1', 'pos', 1.0], а ['a1', 'a0', 'pos', 1.0] отсутствует
            for ver in collection[key]['title']:
              group_dict[ver] = p_type
              count += 1
            for var in collection[key]['variants']:
              if len(var.split(' ')) > 1 and var[-2:] == 'ть':
                group_dict[var] = p_type
                count += 1

              
        except Exception as e:
          pass
  return group_dict

pol_type_dict = [[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]], \
                 [['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]], \
                  [['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]], \
                  [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]],
                 ]

groups_total_dict = make_group(pol_type_dict)
len(groups_total_dict)
# make_group(pol_type_dict)


[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]
[['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]]
[['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]]
[['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]


169

In [0]:
opinion_dict = ['одобрять_V', 'хвалить_V', 'поощрять_V']
def make_sid_embedding(seed_dict, model):
  summ = 0
  try:
    for verb in seed_dict:
      #print(model[verb])
      summ = summ + model[verb]
  except KeyError:
    pass
  vector = summ/len(seed_dict)
  return vector
opinion_embedding = make_sid_embedding(opinion_dict, rv_model)


In [83]:
def get_embedding(verbs_list, model, dim=50):
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(verbs_list)
    total = len(verbs_list)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
            print(vectors)
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    return vector

opinion_embedding = get_embedding(opinion_dict, rv_model)
opinion_embedding

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [80]:
opinion_dict = ['одобрять_V', 'хвалить_V', 'поощрять_V']
def make_sid_embedding(seed_dict, model):
  summ = 0
  try:
    for verb in seed_dict:
      #print(model[verb])
      summ = summ + get_embedding(verb, model)
      print(summ)
  except KeyError:
    pass
  vector = summ/len(seed_dict)
  return vector
opinion_embedding = make_sid_embedding(opinion_dict, rv_model)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [79]:
opinion_embedding

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [78]:
cosine_distances(opinion_embedding, get_embedding(['хвалить_V'], rv_model, dim=50))

ValueError: ignored

In [42]:
def make_similarity_df(base_vect, verbs_for_check_dict, model):
  '''
  Считает косинусное расстояние с каким-либо подающимся на вход сидовым глаголом глаголов из словаря, полученного с помощью 
  функции make_group. Возвращает словарь с ключами 'groups', 'base_verb', 'verb' и 'verbs_similarity', 
  которые потом будут колонками таблицы - датафрейма.
  '''

  df_dict = Counter()
  df_dict['groups'] = []
  df_dict['base_verb'] = []
  df_dict['verb'] = []
  df_dict['verbs_similarity'] = []

  #my_base_verb = base_verb + '_V'
  for verb in verbs_for_check_dict.keys():
    verb_for_check = verb + '_V'
    try:
      verbs_similarity = model.similarity(base_vect, verb_for_check)
      df_dict['groups'].append(str(verbs_for_check_dict[verb]))
      df_dict['base_verb'].append(base_verb)
      df_dict['verb'].append(verb)
      df_dict['verbs_similarity'].append(verbs_similarity)
    except KeyError as e:
      pass
      #print(e)
  return df_dict

verbs_for_df = make_similarity_df(opinion_embedding, groups_total_dict, rv_model)
print('done')

done


In [0]:

frames_df = pd.DataFrame(verbs_for_df, columns = verbs_for_df.keys()) # тут делается датафрейм

In [44]:
frames_df

,groups,base_verb,verb,verbs_similarity
